In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from networkx.algorithms import community as cm

### CONCOR

É um dos primeiros e mais usados métodos para detectar comunidades. 

Ele é baseado na convergência de correlações iteradas.

O método CONCOR aplicado uma única vez, divide o conjunto de nós em no máximo duas comunidades. 
Se mais comunidades forem necessárias, pode-se aplicar novamente o método CONCOR em uma comunidade encontrada anteriormente, dividindo-a em duas.

**Exemplo**

Considerar novamente o exemplo do livro 2 da saga Harry Potter. 

Recorde que as ligações na rede representam suporte emocional e, portanto, trata-se de uma rede direcionada com matriz de adjacência não simétrica.

Usamos o algoritmo **CONCOR** para achar uma divisão em duas comunidades. 

Excluímos os nós isolados. 
Estes vértices podem ser considerados uma comunidade separada, mas alguns deles correspondem a personagens que não fazem parte do livro. 

A convergência do algoritmo é detectada usando a função **allclose** de Numpy porque, trabalhando com aritmética de ponto flutante, existe a possibilidade de que os elementos da matriz $C_k$ não fiquem exatamente iguais a $+-1$. 

As correlações são feitas usando a função **corr_coef** de Numpy. 

A função **adjacency_matrix** devolve a matriz em um formato especial para matrizes esparsas. 

Antes de começar o algoritmo CONCOR, primeiro convertemos a matriz a um formato denso com **todense**.

**Exemplo**

Considere o exemplo da rede dos personagens do segundo livro da saga de Harry Potter.

In [ ]:
atri = pd.read_csv('../data/harrypotter/hpattributes.txt', sep='\t')
ares = pd.read_csv('../data/harrypotter/hpbook2.txt', sep=' ', header=None)
nome = pd.read_csv('../data/harrypotter/hpnames.txt', sep='\t')

In [ ]:
# criando a rede

gpotter = nx.DiGraph()

n = atri.shape[0]

for k in range(n):
    gpotter.add_node(k,
                     nome = nome['name'][k],
                     ano = atri['schoolyear'][k],
                     gen = atri['gender'][k],
                     casa = atri['house'][k])

for k in range(n):
    for m in range(n):
        if ares.values[k][m] == 1:
            gpotter.add_edge(k,m)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10))

nx.draw(gpotter,with_labels=True)

In [ ]:
# Matriz de adjacência esparsa
A = nx . adjacency_matrix ( gpotter )

# Apagamos os nós isolados :
isolados = []
for k in range(n):
    if np.sum(A[[k],:]) == 0:
        isolados.append(k)
        gpotter.remove_node(k)

print(len(isolados), 'nos isolados')

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10))

nx.draw(gpotter,with_labels=True)

In [ ]:
# matriz de adjacencia esparsa
A = nx.adjacency_matrix(gpotter)

# matriz de adjacencia densa
A = np.array(A.todense(),dtype=float)

In [ ]:
# algoritmo CONCOR
C = A.copy()

while not np.allclose(np.abs(C), np.ones_like(C), atol=1e-10):
    C = np.corrcoef(C)

C = np.round(C)

**allclose()**

Returns True if two arrays are element-wise equal within a tolerance.

**ones_like()**

Return an array of ones with the same shape and type as a given array.

 **corrcoef()**

Return Pearson product-moment correlation coefficients.

In [ ]:
# algoritmo CONCOR

com1 = []
com2 = []

nos  = list(gpotter.nodes)

com1.append(nos[0])

for k in range(1,len(nos)):
    if C[0,k] > 0:
        com1.append(nos[k])
    else:
        com2.append(nos[k])

In [ ]:
len(com1)

In [ ]:
len(com2)

In [ ]:
print('Comunidade 1:')
for k in range(len(com1)):
    print(gpotter.nodes[com1[k]]['nome'])

In [ ]:
print('Comunidade 2:')
for k in range(len(com2)):
    print(gpotter.nodes[com2[k]]['nome'])

In [ ]:
plt.close('all')

fig, ax = plt.subplots(1,1,figsize=(15,10))

pos = nx.circular_layout(gpotter)

for k in range(len(com1)):
    pos[com1[k]][0] = 100*np.cos(2*np.pi*k/len(com1))
    pos[com1[k]][1] = 100*np.sin(2*np.pi*k/len(com1))

for k in range(len(com2)):
    pos[com2[k]][0] = 300*np.cos(2*np.pi*k/len(com2))
    pos[com2[k]][1] = 300*np.sin(2*np.pi*k/len(com2))    

cores = []
for no in gpotter.nodes():
    if no in com1:
        cores.append('lightskyblue')
    else:
        cores.append('steelblue')
    
etiquetas = nx.get_node_attributes(gpotter, 'nome')

nx.draw_networkx(gpotter,pos=pos,edge_color='lightgray',
                 node_size = 500, labels=etiquetas,
                 font_size=10, node_color=cores)

plt.box(False)
plt.show()

### Divisão de comunidades

Os problemas de divisão de um grafo em comunidades são, comumente, muito complexos. 

Daí que existem heurísticas para resolver esses problemas.

Os scripts python a seguir minimiza a razão entre o número de arestas que ligam dois grupos e o produto dos tamanhos dos dois grupos, 
aplicado a uma rede de personagens do livro 2 da saga Harry Potter.

O sub-pacote **algorithms.community** do Networkx contém várias funções relacionadas a busca de comunidades em redes.

O primeiro número que devolve a função **partition_quality** é a razão entre o número de arestas internas aos grupos e o número total de arestas. 

In [ ]:
mper = +np.inf
for ini in gpotter.nodes:
    com1 = set([ini])
    com2 = set(nos) - com1
    
    mgan = -1
    while mgan < 0:
        cv, pe = cm.partition_quality(gpotter,[com1,com2])
        ra = (1-cv)/(len(com1)*len(com2)) 
        mgan = +np.inf
        mno  = -1
        for no in com2:
            ncom1 = com1.union(set([no]))
            ncom2 = com2 - set([no])
            cv,pe = cm.partition_quality(gpotter,[ncom1,ncom2])
            nra   = (1-cv)/(len(ncom1)*len(ncom2))
            ng    = nra-ra
            if ng < mgan:
                mgan = ng
                mno  = no
        if mgan < 0:
            com1.add(mno)
            com2.remove(mno)
    
    cv,pe = cm.partition_quality(gpotter,[com1,com2])            
    ra = (1-cv)/(len(com1)*len(com2)) 

    if ra < mper:
        mper = ra
        mcom1 = com1.copy()
        mcom2 = com2.copy()

In [ ]:
com1 = list(mcom1)
com2 = list(mcom2)

In [ ]:
print('Comunidade 1:')
for k in range(len(com1)):
    print(com1[k],gpotter.nodes[com1[k]]['nome'])

In [ ]:
print('Comunidade 2:')
for k in range(len(com2)):
    print(com2[k],gpotter.nodes[com2[k]]['nome'])

In [ ]:
plt.close('all')

fig, ax = plt.subplots(1,1,figsize=(15,10))

pos = nx.circular_layout(gpotter)

for k in range(len(com1)):
    pos[com1[k]][0] = 300*np.cos(2*np.pi*k/len(com1))
    pos[com1[k]][1] = 300*np.sin(2*np.pi*k/len(com1))+700

for k in range(len(com2)):
    pos[com2[k]][0] = 300*np.cos(2*np.pi*k/len(com2))
    pos[com2[k]][1] = 300*np.sin(2*np.pi*k/len(com2))    

cores = []
for no in gpotter.nodes():
    if no in com1:
        cores.append('lightskyblue')
    else:
        cores.append('steelblue')

etiquetas = nx.get_node_attributes(gpotter, 'nome')

nx.draw_networkx(gpotter,
                 pos=pos,
                 edge_color='lightgray',
                 node_size = 500, 
                 labels=etiquetas,
                 font_size=10, 
                 node_color=cores)

plt.box(False)
plt.show()

Um algoritmo de bisseção muito conhecido é o algoritmo de **Kernighan e Lin**. 

Este algoritmo guloso começa com uma partição do grafo (dada ou ao acaso) e prossegue trocando dois nós, um de cada grupo, de maneira de aprimorar a partição.

A seguir aplicamos o algoritmo **Kernighan e Lin** a uma rede de personagens do livro 2 da saga Harry Potter.

**cm.kernighan_lin_bisection()**

Partition a graph into two blocks using the **Kernighan–Lin** algorithm.

This algorithm partitions a network into two sets by iteratively swapping pairs of nodes to reduce the edge cut between the two sets. 

The pairs are chosen according to a modified form of **Kernighan-Lin**, which moves node individually, alternating between sides to keep the bisection balanced.

In [ ]:
# Algoritmo Kernighan e Lin aplicado a uma rede de personagens do segundo livro da saga Harry Potter.
com1, com2 = cm.kernighan_lin_bisection(gpotter.to_undirected(), max_iter=100)

In [ ]:
print("com1:",com1)
print("com2:",com2)

In [ ]:
print('Comunidade 1:')
for k in range(len(com1)):
    print(com1[k],gpotter.nodes[com1[k]]['nome'])

In [ ]:
print('Comunidade 2:')
for k in range(len(com2)):
    print(com2[k],gpotter.nodes[com2[k]]['nome'])

In [ ]:
com1 = list(com1)
com2 = list(com2)

In [ ]:
plt.close('all')

fig, ax = plt.subplots(1,1,figsize=(15,10))

pos = nx.circular_layout(gpotter)

for k in range(len(com1)):
    pos[com1[k]][0] = 300*np.cos(2*np.pi*k/len(com1))
    pos[com1[k]][1] = 300*np.sin(2*np.pi*k/len(com1))+700

for k in range(len(com2)):
    pos[com2[k]][0] = 300*np.cos(2*np.pi*k/len(com2))
    pos[com2[k]][1] = 300*np.sin(2*np.pi*k/len(com2))    

cores = []
for no in gpotter.nodes():
    if no in com1:
        cores.append('lightskyblue')
    else:
        cores.append('steelblue')

etiquetas = nx.get_node_attributes(gpotter, 'nome')

#plt.figure()
nx.draw_networkx(gpotter,pos=pos,edge_color='lightgray',
                 node_size = 500, labels=etiquetas,
                 font_size=10, node_color=cores)
plt.box(False)
plt.show()